# DATA TRANSFORMATION - 19_consolidacion
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [1]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf
import adherence_functions as af
import warnings
warnings.filterwarnings('ignore')

ruta_archivos = 'C:/Users/monic/documentos/ds4a/project/datos/'

## 1. Varios

In [2]:
calculos = gf.diccionario_agg_functions()
dcc = gf.diccionario_llaves()

In [3]:
# execute initial file
files = 'C:/Users/monic/documentos/ds4a/ds4a_team80/1_data_transformation/1_consolidacion_mensual/'
file_0 = files + '0_initial.py'

exec(open(file_0).read())

## 2. Cálculo de Variables por Módulo

### 2.1. Funciones

In [4]:
##################################################################
# Funciones de apoyo para el cálculo de las variables
##################################################################
def renombre_variables_nmeses( bx, calculo, nmeses ):
    sufi = '_' + calculo + '_' + str(nmeses)
    bx.columns = [ s + sufi for s in bx.columns]
    bx.rename( columns={ 'id' + sufi :'id','year_obs' + sufi :'year_obs', 'month_obs' + sufi :'month_obs' }, inplace=True)
    
    return bx

def renombre_variables( bx, calculo ):
    sufi = '_' + calculo
    bx.columns = [ s + sufi for s in bx.columns]
    bx.rename( columns={ 'id' + sufi :'id','year_obs' + sufi :'year_obs', 'month_obs' + sufi :'month_obs' }, inplace=True)
    
    return bx

def cruza_adfeat( bx, base_ad_features):
    bx = base_ad_features.merge( bx , how = 'left', 
                                 left_on = ['id','year','month'], right_on = ['id','year_obs','month_obs'] )
    bx = bx.drop(['year_obs', 'month_obs'], axis=1)
    return bx

##################################################################
# Funciones que realizan el cálculo de las variables
##################################################################
def variables_sum( variables, idx, base_1, calculo, nmeses ):
    bx = base_1.groupby( idx )[variables].sum().reset_index()
    
    bx = renombre_variables_nmeses( bx, calculo, nmeses )
    return bx

def variables_avg( variables, idx, base_1, calculo, nmeses ):
    bx = base_1.groupby( idx )[variables].mean().reset_index()
    bx = renombre_variables_nmeses( bx, calculo, nmeses )
    
    return bx

def variables_foto( variables, base_cruce, calculo ):
    # Limita el número de meses al último mes
    bx = base_cruce[base_cruce['d_meses'] == 1]
    bx = bx[ variables]    
    bx = renombre_variables( bx, calculo )
    
    return bx

def variables_flag( variables, idx, base_cruce, calculo ):
    # No limita el número de meses
    bx = base_cruce
    bx = bx.groupby( idx )[variables].max().reset_index()
    bx = renombre_variables( bx, calculo )
    
    return bx

In [9]:
# Para el cálculo de variables sum y avg a nmeses
def cal_variables( base_features, base_ad_12, idx, nmeses, calculos, cc, dc_n, base_ad_features  ):
    
    # Cruza con adherencia
    base_cruce = base_ad_12.merge( base_features , how = 'inner', 
                            left_on=['id','year_c','month_c'], right_on = ['id','year','month'] )
    
    # Limita el número de meses
    base_1 = base_cruce[base_cruce['d_meses'] <= nmeses]
    
    # Realiza agrupaciones de acuerdo al cálculo
    for calculo in calculos[ cc ]:
        
        variables = calculos[ cc ][calculo]
        
        if calculo == 'sum' :
            
            #print( "Ejecutando modulo " + cc  + " y calculo " + calculo)
            
            # Realiza el cálculo
            base_12 = variables_sum( variables, idx, base_1, calculo, nmeses )
            
            # Guarda base
            dc_n[cc][ calculo + '_' + str(nmeses) ] = base_12
            
            # Cruza con base total
            base_ad_features = cruza_adfeat( base_12, base_ad_features)
            
            
        elif calculo == 'avg' :
            
            #print( "Ejecutando modulo " + cc  + " y calculo " + calculo)
            
            # Realiza el cálculo
            base_12 = variables_avg( variables, idx, base_1, calculo, nmeses )
            
            # Guarda base
            dc_n[cc][ calculo + '_' + str(nmeses) ] = base_12
            
            # Cruza con base total
            base_ad_features = cruza_adfeat( base_12, base_ad_features)        
            
    return dc_n, base_ad_features


In [11]:
# Para el cálculo de variables foto o flag

def cal_variables_f( base_features, base_ad_12, idx, calculos, cc, dc_n, base_ad_features  ):
    
    # Cruza con adherencia
    base_cruce = base_ad_12.merge( base_features , how = 'inner', 
                            left_on=['id','year_c','month_c'], right_on = ['id','year','month'] )
    
    # Realiza agrupaciones de acuerdo al cálculo
    for calculo in calculos[ cc ]:    
        
        variables = calculos[ cc ][calculo] 
        
        if calculo == 'foto' :
            
            # print( "Ejecutando modulo " + cc  + " y calculo " + calculo)
            
            # Realiza el cálculo
            variables = variables + idx
            base_12 = variables_foto( variables, base_cruce, calculo )
            
            # Guarda base
            dc_n[cc][ calculo ] = base_12
            
            # Cruza con base total
            base_ad_features = cruza_adfeat( base_12, base_ad_features)

            
        elif calculo == 'flag' :
            
            # print( "Ejecutando modulo " + cc  + " y calculo " + calculo)
            
            # Realiza el cálculo
            variables = variables
            base_12 = variables_flag( variables, idx, base_cruce, calculo )
            
            # Guarda base
            dc_n[cc][ calculo ] = base_12
            
            # Cruza con base total
            base_ad_features = cruza_adfeat( base_12, base_ad_features)
        
    return dc_n, base_ad_features


### 2.2. Bases Iniciales

In [7]:
base_ini = gf.base_ids_mensual( ruta_archivos )
_ , base_ad_0 = af.base_features_adeherencia0( ruta_archivos, dcc, ids )
base_ad_12, base_ad = af.base_features_adeherencia_meses( ruta_archivos, dcc, ids , 12)

### 2.3 Ejecución de Variables

In [12]:
idx = ['id', 'year_obs', 'month_obs']
nmeses = [6,12]
dc_n = {}

base_ad_features = base_ad

for cc in calculos :
    
    # Ejecuta el modulo
    if cc == 'adh' :
        base_i = base_ad_0[['id','year','month','adeherencia_0']]
        base_i.columns =  ['id','year','month','adh']
        print( "Ejecutando modulo " + cc )
    else :
        m = calculos[cc]['mod']
        file_i = files + dcc[ m ]['prefi'] + '.py'
        exec(open(file_i).read())
        exec("base_i = base_final_" + dcc[m]['prefi'] )
        print( "Ejecutando modulo " + cc )
    
    # Quita los nulls
    cols_i = base_i.columns

    base_i['nuls'] = base_i.isnull().sum(axis = 1)
    todo_null = base_i.shape[1] - 4    #id, month, year, nuls
    base_features = base_i[base_i['nuls'] < todo_null ]
    base_features.reset_index( inplace = True )
    base_features = base_features[cols_i]
    #print(base_features.shape)
    
    # Calcula las variables
    dc_n[ cc ] = {}
    
    # Calculos sum, avg
    for mn in nmeses : 
        dc_n, base_ad_features = cal_variables( base_features, base_ad_12, idx, mn, calculos, cc, dc_n, base_ad_features  )
        
    # Calculos foto
    dc_n, base_ad_features = cal_variables_f( base_features, base_ad_12, idx, calculos, cc, dc_n, base_ad_features )
    
    print(base_ad_features.shape)


Ejecutando modulo far
(3027, 32)
Ejecutando modulo hos
(3027, 48)
Ejecutando modulo med
(3027, 60)
Ejecutando modulo urg
(3027, 68)
Ejecutando modulo vac
(3027, 71)
Ejecutando modulo cal
(3027, 83)
Ejecutando modulo act
(3027, 85)
Ejecutando modulo acd
(3027, 95)
Ejecutando modulo ant
(3027, 99)
Ejecutando modulo anf
(3027, 103)
Ejecutando modulo ayu
(3027, 105)
Ejecutando modulo bio
(3027, 117)
Ejecutando modulo epo
(3027, 127)
Ejecutando modulo hab
(3027, 133)
Ejecutando modulo inc
(3027, 137)
Ejecutando modulo imc
(3027, 140)
Ejecutando modulo adh
(3027, 142)


In [13]:
base_ad_features.to_csv( 'base_adeherencia_features_1.csv', sep='|', index = False )

-----------------------------------------------------------------------------

In [20]:
base_ad_0.head()

,id,year,month,adeherencia_0,key,nuls
0,500547,2016,7,0.0,1,0
1,500547,2019,3,0.0,1,0
2,500547,2019,10,0.0,1,0
3,500547,2019,12,0.0,1,0
4,500547,2020,1,0.0,1,0


In [9]:
print(base_ad_features.shape)

cols_i = base_ad_features.columns

base_ad_features['nuls'] = base_ad_features.isnull().sum(axis = 1)
todo_null = base_ad_features.shape[1] - 6    #id, month, year, adh, key, nuls
base_ad_features = base_ad_features[base_ad_features['nuls'] < todo_null ]
base_ad_features.reset_index( inplace = True )
base_ad_features = base_ad_features[cols_i]
base_ad_features.shape

(5085, 139)


(3663, 151)

In [13]:
base_ad_features.head(10)

,id,year,month,adeherencia_0,key,far_rea_asma_sum_6,far_rea_total_sum_6,far_evo_aun_sum_6,far_cau_def_sum_6,far_cau_otra_sum_6,...,hab_alcohol_avg_12,hab_cigarrillo_avg_12,hab_ejercicio_avg_12,inc_con_sum_6,inc_inc_sum_6,inc_con_sum_12,inc_inc_sum_12,imc_avg_6,imc_avg_12,imc_foto
0,500547,2016,7,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,3.0,0.0,3.0,NaN,NaN,NaN
1,500547,2019,3,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,500547,2019,10,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.800,NaN
3,500547,2019,12,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.800,NaN
4,500547,2020,1,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.800,NaN
5,500547,2020,3,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.800,NaN
6,500547,2020,5,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,500588,2017,8,0,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,500588,2019,4,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,2.0,16.618,16.794,16.26
9,502989,2017,1,0,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,4.0,0.0,4.0,NaN,NaN,NaN


In [11]:
base_ad_features.groupby(['year','adeherencia_0'])['id'].count()

year  adeherencia_0
2016  0                 98
      1                329
2017  0                244
      1                654
2018  0                293
      1                709
2019  0                207
      1                781
2020  0                 30
      1                318
Name: id, dtype: int64

## Revisiones

In [12]:
# Parameters
modulo = 'Mediciones de peso y talla'

# Extract
base = gf.carga_datos( ruta = ruta_archivos, diccionario = dcc, modulo = modulo )

# Transform

base.rename(columns={'NM_IMC': dcc[modulo]['prefi']}, inplace=True)
# Elimina dplicados
base = base.drop_duplicates()
# Elimina columnas sobran
base = base.drop(columns=['NM_PESO','NM_TALLA','Clasificacion_IMC'])

base = base.drop(columns=['year_month', 'fecha'])


# Merge
base_final_imc = ids_mensual.merge( base, how='left')

,id,year,month,imc
0,500547,2019,3,25.80
1,500588,2017,8,16.22
2,500588,2017,9,17.01
3,500588,2017,10,17.80
4,500588,2017,11,17.97
...,...,...,...,...
9854,1999536,2019,7,22.03
9855,1999536,2019,8,21.63
9856,1999536,2019,9,21.63
9857,1999629,2018,3,16.19


In [8]:
base_ad_12[ ( base_ad_12['id']==500547) & ( base_ad_12['year_obs']==2019) & ( base_ad_12['month_obs']==3) ].head(5)

,id,year_obs,month_obs,adeherencia_0,key,year_c,month_c,d_meses
60,500547,2019,3,1,1,2016,1,38
61,500547,2019,3,1,1,2016,2,37
62,500547,2019,3,1,1,2016,3,36
63,500547,2019,3,1,1,2016,4,35
64,500547,2019,3,1,1,2016,5,34


In [9]:
base_ad[ ( base_ad['id']==500547) & ( base_ad['year']==2019) & ( base_ad['month']==3) ]

,id,year,month,adeherencia_0,key
1,500547,2019,3,1,1


In [10]:
print(base_ini.shape)
base_ini.head()

(52320, 3)


,id,year,month
0,500547,2016,1
1,500547,2016,2
2,500547,2016,3
3,500547,2016,4
4,500547,2016,5
